# APEX Gun, IMPACT-T

In [ ]:
from impact import Impact
from distgen import Generator

import os

# Nicer plotting
import matplotlib.pyplot as plt
import matplotlib

%config InlineBackend.figure_format = 'retina'
matplotlib.rcParams["figure.figsize"] = (8, 4)

In [ ]:
IMPACT_IN = "../templates/apex_gun/ImpactT.in"
DISTGEN_IN = "../templates/apex_gun/distgen.yaml"
NUMPROCS = 0
os.path.exists(IMPACT_IN)

In [ ]:
G = Generator(DISTGEN_IN)
G["n_particle"] = 10000

G.run()
P0 = G.particles
factor = 1.01
P0.x *= factor
P0.y *= 1 / factor
P0.plot("x", "y")

In [ ]:
P0["charge"]

In [ ]:
# Make Impact object
I = Impact(IMPACT_IN)

In [ ]:
I.track1()
I.plot("mean_kinetic_energy")

In [ ]:
I.initial_particles = P0

# Change some things
I.header["Nx"] = 32
I.header["Ny"] = 32
I.header["Nz"] = 32
I.header["Dt"] = 1e-13

# I.total_charge = P0['charge']
I.total_charge = 0  # Turn off space charge

# Other switches
I.timeout = 1000

# Change stop location
I.stop = 0.15

# Switches for MPI
I.numprocs = NUMPROCS

In [ ]:
%%time
I.run()
print(I)

# Particles

In [ ]:
# Particles are automatically parsed in to openpmd-beamphysics ParticleGroup objects
I.output["particles"]

In [ ]:
# Get the final particles, calculate some statistic
P = I.output["particles"]["final_particles"]
P["mean_energy"]

In [ ]:
# Show the units
P.units("mean_energy")

In [ ]:
P.plot("z", "pz")

# Stats

In [ ]:
# Impact's own calculated statistics can be retieved
len(I.stat("norm_emit_x")), I.stat("norm_emit_x")[-1]

In [ ]:
# Stats can also be computed from the particles. For example:
I.particles["final_particles"]["norm_emit_x"]

In [ ]:
# Compare these.
key1 = "mean_z"
key2 = "sigma_x"
units1 = str(I.units(key1))
units2 = str(I.units(key2))
plt.xlabel(key1 + f" ({units1})")
plt.ylabel(key2 + f" ({units2})")
plt.plot(I.stat(key1), I.stat(key2))
plt.scatter(
    [I.particles[name][key1] for name in I.particles],
    [I.particles[name][key2] for name in I.particles],
    color="red",
)